In [ ]:
import json
import os
import shutil
import random
import copy
from pandas.core.common import flatten
import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance
import cv2
import albumentations as A

In [ ]:
in_dir = "/mnt/nis_lab_research/data/coco_files/aug/far_shah-b1-b2_cln_EOI/"

In [ ]:
out_dir = "/mnt/nis_lab_research/data/coco_files/aug/far_shah-b1-b2_cln_EOI_aug_all/"

In [ ]:
# Setting augmentation types
# Value is number of images of each type generated
# A value of 0 is off

adj_brightness = 1
adj_contrast = 1
adj_saturation = 1
adj_hue = 1
adj_color_jitter = 1
adj_posterize = 1
adj_blur = 1




In [ ]:
def getRandTup(low_low, low_high, high_low, high_high):

    rand_low = random.uniform(low_low,low_high)
    rand_high = random.uniform(high_low, high_high)
    rand_pick = random.uniform(0,1)
    
    rand_use = 0
    
    if rand_pick < .5:
        rand_use = rand_low
    else:
        rand_use = rand_high
        
    rand_tup = (rand_use, rand_use)
    
    return rand_tup



In [ ]:
def aug_brightness(og_img):
        
    rand_tup = getRandTup(.3, .8, 2, 3.5)
        
    aug = A.ColorJitter(brightness=rand_tup,contrast=0,saturation=0,hue=0, always_apply=True, p=1)
    aug_img =  aug(image=og_img)['image']
    return aug_img



In [ ]:
def aug_contrast(og_img):
    
    rand_tup = getRandTup(.3, .8, 2, 3.5)
        
    aug = A.ColorJitter(brightness=0,contrast=rand_tup,saturation=0,hue=0, always_apply=True, p=1)
    aug_img =  aug(image=og_img)['image']
    return aug_img

In [ ]:
def aug_saturation(og_img):
    
    rand_tup = getRandTup(.3, .8, 2, 3.5)
        
    aug = A.ColorJitter(brightness=0,contrast=0,saturation=rand_tup,hue=0, always_apply=True, p=1)
    aug_img =  aug(image=og_img)['image']
    return aug_img

In [ ]:
def aug_hue(og_img):
    
    rand_tup = getRandTup(-.5, -.075, .075, .5)
        
    aug = A.ColorJitter(brightness=0,contrast=0,saturation=0,hue=rand_tup, always_apply=True, p=1)
    aug_img =  aug(image=og_img)['image']
    return aug_img

In [ ]:
def aug_color_jitter(og_img):
        
    aug = A.ColorJitter(brightness=getRandTup(.3, .8, 1.5, 2.25),
                        contrast=getRandTup(.3, .8, 1.5, 2.25),
                        saturation=getRandTup(.3, .8, 1.5, 2.25),
                        hue=getRandTup(-.5, -.075, .075, .5), 
                        always_apply=True, p=1)
    
    aug_img =  aug(image=og_img)['image']
    return aug_img

In [ ]:
def aug_posterize(og_img):
    
    aug = A.Posterize(num_bits=(1,2), always_apply=True, p=1)
    
    aug_img =  aug(image=og_img)['image']
    return aug_img

In [ ]:
def aug_blur(og_img):
    
    aug = A.Blur(blur_limit=(5, 9), always_apply=True, p=1)
    
    aug_img =  aug(image=og_img)['image']
    return aug_img

In [ ]:
def createImgJson (img_id, fn):
    
    json_obj = {
        'width': 1920,
        'height': 1080,
        'id': img_id,
        'file_name': fn
    }
    
    return json_obj

In [ ]:
def createAnnJson (ann_id, img_id, cat_id, seg, bbox, ignore, iscrowd, area):
    
    json_obj = {
        'id': ann_id,
        'image_id': img_id,
        'category_id': cat_id,
        'segmentation': seg,
        'bbox': bbox,
        'ignore': ignore,
        'iscrowd': iscrowd,
        'area': area
    }
    
    return json_obj

In [ ]:
with open(os.path.join(in_dir, "result.json")) as f:
    coco_obj = json.load(f)

In [ ]:
img_list_og = coco_obj["images"]
ann_list_og = coco_obj["annotations"]
cat_list_og = coco_obj["categories"] 

In [ ]:
img_list_master = []
ann_list_master = []

img_id_new = 0
ann_id_new = 0

for i, img in enumerate(img_list_og):
    
    fn_rel = img["file_name"]
    fn = fn_rel[2:-4]
    fn_full = os.path.join(in_dir, "images", fn_rel[2:])
    img_id_og = img["id"]
    
    print (str(i) + ")", fn_full)
    
    # Creating directory for images inside of coco output file
    img_out_dir = os.path.join(out_dir, "images")
    os.makedirs(img_out_dir, exist_ok=True)
    
    # Collecting og annotations json
    ann_og = []
    for ann in ann_list_og:
        if img_id_og == ann["image_id"]:
            ann_og.append(ann)  
    
    # Handling og images 
    fn_new = fn + "-og-" + str(img_id_new) + ".png"
    og_img_path = os.path.join(img_out_dir, fn_new)
    og_img = cv2.imread(fn_full)
    cv2.imwrite(og_img_path, og_img)
    
    # Handling og image json
    img_list_master.append(createImgJson(img_id_new, "./" + fn_new))
    
    # Handling og annotations json
    for ann in ann_og:
        ann_new = createAnnJson(ann_id_new, img_id_new, ann["category_id"], ann["segmentation"], ann["bbox"], ann["ignore"], ann["iscrowd"], ann["area"])
        ann_list_master.append(ann_new)
        ann_id_new += 1
    
    img_id_new += 1
    
    ann_new = []
    
    if adj_brightness > 0:
        for j in range(0, adj_brightness):
            fn_new = fn + "-bright-" + str(img_id_new) + ".png"
            bright_img_path = os.path.join(img_out_dir, fn_new)
            bright_img = aug_brightness(og_img)
            cv2.imwrite(bright_img_path, bright_img)
            
            img_list_master.append(createImgJson(img_id_new, "./" + fn_new))
            
            for ann in ann_og:
                ann_new = createAnnJson(ann_id_new, img_id_new, ann["category_id"], ann["segmentation"], ann["bbox"], ann["ignore"], ann["iscrowd"], ann["area"])
                ann_list_master.append(ann_new)
                ann_id_new += 1
            
            img_id_new += 1
            
    if adj_contrast > 0:
        for j in range(0, adj_contrast):
            fn_new = fn + "-contrast-" + str(img_id_new) + ".png"
            contrast_img_path = os.path.join(img_out_dir, fn_new)
            contrast_img = aug_contrast(og_img)
            cv2.imwrite(contrast_img_path, contrast_img)
            
            img_list_master.append(createImgJson(img_id_new, "./" + fn_new))
            
            for ann in ann_og:
                ann_new = createAnnJson(ann_id_new, img_id_new, ann["category_id"], ann["segmentation"], ann["bbox"], ann["ignore"], ann["iscrowd"], ann["area"])
                ann_list_master.append(ann_new)
                ann_id_new += 1
            
            img_id_new += 1
    
    if adj_saturation > 0:
        for j in range(0, adj_saturation):
            fn_new = fn + "-saturate-" + str(img_id_new) + ".png"
            saturate_img_path = os.path.join(img_out_dir, fn_new)
            saturate_img = aug_saturation(og_img)
            cv2.imwrite(saturate_img_path, saturate_img)
            
            img_list_master.append(createImgJson(img_id_new, "./" + fn_new))
            
            for ann in ann_og:
                ann_new = createAnnJson(ann_id_new, img_id_new, ann["category_id"], ann["segmentation"], ann["bbox"], ann["ignore"], ann["iscrowd"], ann["area"])
                ann_list_master.append(ann_new)
                ann_id_new += 1
            
            img_id_new += 1

    if adj_hue > 0:
        for j in range(0, adj_hue):
            fn_new = fn + "-hue-" + str(img_id_new) + ".png"
            hue_img_path = os.path.join(img_out_dir, fn_new)
            hue_img = aug_hue(og_img)
            cv2.imwrite(hue_img_path, hue_img)
            
            img_list_master.append(createImgJson(img_id_new, "./" + fn_new))
            
            for ann in ann_og:
                ann_new = createAnnJson(ann_id_new, img_id_new, ann["category_id"], ann["segmentation"], ann["bbox"], ann["ignore"], ann["iscrowd"], ann["area"])
                ann_list_master.append(ann_new)
                ann_id_new += 1
            
            img_id_new += 1

    if adj_color_jitter > 0:
        for j in range(0, adj_color_jitter):
            fn_new = fn + "-color_jitter-" + str(img_id_new) + ".png"
            color_jitter_img_path = os.path.join(img_out_dir, fn_new)
            color_jitter_img = aug_color_jitter(og_img)
            cv2.imwrite(color_jitter_img_path, color_jitter_img)
            
            img_list_master.append(createImgJson(img_id_new, "./" + fn_new))
            
            for ann in ann_og:
                ann_new = createAnnJson(ann_id_new, img_id_new, ann["category_id"], ann["segmentation"], ann["bbox"], ann["ignore"], ann["iscrowd"], ann["area"])
                ann_list_master.append(ann_new)
                ann_id_new += 1
            
            img_id_new += 1
            
    if adj_posterize > 0:
        for j in range(0, adj_posterize):
            fn_new = fn + "-posterize-" + str(img_id_new) + ".png"
            posterize_img_path = os.path.join(img_out_dir, fn_new)
            posterize_img = aug_posterize(og_img)
            cv2.imwrite(posterize_img_path, posterize_img)
            
            img_list_master.append(createImgJson(img_id_new, "./" + fn_new))
            
            for ann in ann_og:
                ann_new = createAnnJson(ann_id_new, img_id_new, ann["category_id"], ann["segmentation"], ann["bbox"], ann["ignore"], ann["iscrowd"], ann["area"])
                ann_list_master.append(ann_new)
                ann_id_new += 1
            
            img_id_new += 1
            
    if adj_blur > 0:
        for j in range(0, adj_blur):
            fn_new = fn + "-blur-" + str(img_id_new) + ".png"
            blur_img_path = os.path.join(img_out_dir, fn_new)
            blur_img = aug_blur(og_img)
            cv2.imwrite(blur_img_path, blur_img)
            
            img_list_master.append(createImgJson(img_id_new, "./" + fn_new))
            
            for ann in ann_og:
                ann_new = createAnnJson(ann_id_new, img_id_new, ann["category_id"], ann["segmentation"], ann["bbox"], ann["ignore"], ann["iscrowd"], ann["area"])
                ann_list_master.append(ann_new)
                ann_id_new += 1
            
            img_id_new += 1
            
    

In [ ]:
# Creating json out obj

json_out_obj = {
    "images": img_list_master,
    "categories": cat_list_og,
    "annotations": ann_list_master
}

In [ ]:
# Writing out object
jstr = json.dumps(json_out_obj, indent=4)
with open(os.path.join(out_dir, "result.json"), "w+") as f:
    f.write(jstr)
    